# Pre-Processing

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import h5py
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential 
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Reshape, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import KFold

In [ ]:
# mount your drive, may need to force remount if not working
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# for this to work, first upload the .h5 file into your drive and copy the path, then change the path name variable 
fred_path = '/content/drive/MyDrive/MNIST_synthetic.h5'
will_path = '/content/drive/MyDrive/MNIST_synthetic.h5'
jess_path = '/content/drive/MyDrive/School/COMP 551/COMP 551 Mini Project 3/MNIST_synthetic.h5'

with h5py.File(fred_path, 'r') as hdfid:
     print(hdfid.keys())

     test_dataset  = np.array(hdfid['test_dataset'][()])
     train_dataset  = np.array(hdfid['train_dataset'][()])
     train_labels  = np.array(hdfid['train_labels'][()])

train_onehot_labels = to_categorical(train_labels)
# reshaping for visualization 
train = train_dataset.reshape(train_dataset.shape[0], 64, 64)
test = test_dataset.reshape(test_dataset.shape[0], 64, 64)

train.shape, test.shape, train_dataset.shape, test_dataset.shape

<KeysViewHDF5 ['test_dataset', 'train_dataset', 'train_labels']>


((56000, 64, 64), (14000, 64, 64), (56000, 64, 64, 1), (14000, 64, 64, 1))

In [ ]:
# Pre-processing to get cropped images
# cutting off the top and bottom
new_cut = []
for datapoint in train: 
  new_cut.append(datapoint[20:40,:])

new_cut_test = []
for datapoint in test: 
  new_cut_test.append(datapoint[20:40,:])

new_cut = np.array(new_cut)
new_cut_test = np.array(new_cut_test)

new_cut = new_cut.reshape(new_cut.shape[0], 20, 64, 1)
new_cut_test = new_cut_test.reshape(new_cut_test.shape[0], 20, 64, 1)

# Helper Functions

In [ ]:
# from softmax probabilities to class predictions of form [0, 0, 0, 0, 0]
def evaluate(yhat):
  results = []
  for y in yhat:
    current_datapoint = []
    for val in y:
      current_datapoint.append(np.argmax(val))
    results.append(current_datapoint)
  return results

In [ ]:
# creates output csv with 'id' and 'label' columns
# include path name of file you want to save it in 
def id_label_df(yhat, file_path):
  results = evaluate(yhat)
  df = pd.DataFrame(results, columms = ['Label'])
  df.to_csv(file_path + '/out.csv')

In [ ]:
# k fold validation
def kfold_evaluation(model_fn, input_shape, data, labels, lr=0.01, beta_1=0.9, beta_2=0.99, batch_size=32, n_epochs=10, n_folds=5):
  accuracies = []
  model_save = []
  kfold = KFold(n_folds, shuffle=True, random_state=1)

  for train_idx, val_idx in kfold.split(data):
    model = model_fn(input_shape, lr=lr, beta_1=beta_1, beta_2=beta_2)
    x_train, y_train, x_val, y_val = data[train_idx], labels[train_idx], data[val_idx], labels[val_idx]
    history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_data=(x_val, y_val))
    _, acc = model.evaluate(x_val, y_val, verbose=0)
    print('Fold accuracy: ', acc*100.0)
    # print('Mean accuracy:', np.mean(accuracy))
    accuracies.append(acc)
    model_save.append(history)
  return np.mean(accuracies), accuracies, model_save

In [ ]:
# k fold training
def kfold_training(model_fn, input_shape, data, labels, batch_size=32, n_epochs=10, n_folds=5):
  accuracies = []
  model_save = []
  kfold = KFold(n_folds, shuffle=True, random_state=1)
  model = model_fn(input_shape)

  for train_idx, val_idx in kfold.split(data):
    x_train, y_train, x_val, y_val = data[train_idx], labels[train_idx], data[val_idx], labels[val_idx]
    history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_data=(x_val, y_val))
    accuracy = history.history['val_accuracy']
    _, acc = model.evaluate(x_val, y_val, verbose=0)
    print('Fold accuracy: ', acc*100.0)
    print('Mean val accuracy:', accuracy)
    accuracies.append(np.mean(accuracy))
    model_save.append(history)
  return model, accuracies, model_save

In [ ]:
# plots accuracy and loss curves for kfold
def learning_curves_kfold(model_saves, string):
  fig = plt.figure(figsize=(7,3))
  fig.suptitle('Loss and Validation Accuracy Curves for \n' + string, y=1.1)
  # plt.xlabel('Epoch')
  for i in range(len(model_saves)):
    ax_1 = fig.add_subplot(1, 2, 1)
    ax_1.set_title('Binary Cross Entropy Loss')
    ax_1.plot(model_saves[i].history['loss'], color='blue', label='train')
    ax_1.plot(model_saves[i].history['val_loss'], color='orange', label='validation')
    ax_1.set_xticks(range(0,10))
    
    ax_2 = fig.add_subplot(1, 2, 2)
    ax_2.set_title('Validation Accuracy')
    ax_2.plot(model_saves[i].history['accuracy'], color='blue', label='train')
    ax_2.plot(model_saves[i].history['val_accuracy'], color='orange', label='validation')
    ax_2.set_xticks(range(0,10))
  plt.xlabel('Epoch')
  plt.show()

In [ ]:
# plots accuracy and loss curves
def learning_curves(model_saves):
  plt.subplot(2, 1, 1)
  plt.title('Binary Cross Entropy Loss')
  plt.plot(model_saves.history['loss'], color='blue', label='train')
  plt.plot(model_saves.history['val_loss'], color='orange', label='validation')
  
  plt.subplot(2, 1, 2)
  plt.title('Classification Accuracy')
  plt.plot(model_saves.history['accuracy'], color='blue', label='train')
  plt.plot(model_saves.history['val_accuracy'], color='orange', label='validation')
  plt.tight_layout()

In [ ]:
# box plot of accuracy with std
def performance_plot(scores):
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	plt.boxplot(scores)
	plt.tight_layout()

In [ ]:
## IMAGE AUGMENTATION
datagen1 = ImageDataGenerator(shear_range=0.3)
datagen2 = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

adaptive_lr = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

In [ ]:
def results_to_csv_string(results, filename):
  as_strings = []
  for r in results:
    a_s = [str(i) for i in r]
    s = ''.join(a_s)
    as_strings.append(s)

  submissions = pd.Series(as_strings, name="Label")
  submission = pd.concat([pd.Series(range(0, 14000), name="Id"), submissions], axis=1)
  submission.to_csv('/content/drive/MyDrive/' + filename + '.csv', index=False)

# Original Model

In [ ]:
def model(dropout=0.4, input_shape=(64,64,1)):
  model = Sequential()
  model.add(Conv2D(32, kernel_size= 3, padding='same', activation='relu', input_shape = input_shape))
  model.add(BatchNormalization())
  model.add(Conv2D(32, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(32, kernel_size= 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout))

  model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size= 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout))

  model.add(Conv2D(128, kernel_size = 4, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dropout(dropout))
  model.add(Dense(55, activation='softmax'))
  model.add(Reshape((5, 11)))

  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
  return model

In [ ]:
# original model - original data
original_model = model(0.4)

train_x, test_x, train_y, test_y = train_test_split(train_dataset, train_onehot_labels, shuffle=True, test_size=0.2)
history = original_model.fit(train_x, train_y, epochs=10, batch_size=32, validation_data=(test_x, test_y), verbose=1)

predictions = original_model.predict(test_dataset)
result = evaluate(predictions)

results_to_csv(result, 'original_model-original_data')

In [ ]:
# original model - sheared data
original_model = model(0.4)

train_x, test_x, train_y, test_y = train_test_split(train_dataset, train_onehot_labels, shuffle=True, test_size=0.2)
history = original_model.fit(datagen1.flow(train_x, train_y, batch_size=32), epochs=10, steps_per_epoch=train_x.shape[0]//32,
                             batch_size=32, validation_data=(test_x, test_y), verbose=1)

predictions = original_model.predict(test_dataset)
result = evaluate(predictions)

results_to_csv(result, 'original_model-sheared_data')

In [ ]:
# original model - cropped data
original_model = model(0.4, (20,64,1))

train_x, test_x, train_y, test_y = train_test_split(new_cut, train_onehot_labels, shuffle=True, test_size=0.2)
history = original_model.fit(train_x, train_y, epochs=10,
                             batch_size=32, validation_data=(test_x, test_y), verbose=1)

predictions = original_model.predict(new_cut_test)
result = evaluate(predictions)

results_to_csv(result, 'original_model-cropped data')

In [ ]:
# original model - aug2 data
original_model = model(0.4)

train_x, test_x, train_y, test_y = train_test_split(train_dataset, train_onehot_labels, shuffle=True, test_size=0.2)
history = original_model.fit(datagen2.flow(train_x, train_y, batch_size=32), epochs=10, steps_per_epoch=train_x.shape[0]//32,
                             batch_size=32, validation_data=(test_x, test_y), verbose=1)

predictions = original_model.predict(test_dataset)
result = evaluate(predictions)

results_to_csv(result, 'original_model-aug2_data')

In [ ]:
# original model - adaptive learning rate
original_model = model(0.4)

train_x, test_x, train_y, test_y = train_test_split(train_dataset, train_onehot_labels, shuffle=True, test_size=0.2)
history = original_model.fit(train_x, train_y, epochs=10, batch_size=32, validation_data=(test_x, test_y), callbacks=[adaptive_lr], verbose=1)

predictions = original_model.predict(test_dataset)
result = evaluate(predictions)

results_to_csv(result, 'original_model-aug2_data')

In [ ]:
model_4 = original_model(0.4, (20, 64, 1))
train_x, test_x, train_y, test_y = train_test_split(new_cut, train_onehot_labels, shuffle=True, test_size=0.2)
history = model_4.fit(datagen1.flow(train_x, train_y, batch_size=32), epochs=5, validation_data=(datagen.flow(test_x, test_y, batch_size=32)))

predictions = model_4.predict(new_cut_test)
result = evaluate(predictions)

results_to_csv(result, 'result_shear_cut')

In [ ]:
# Dropout test
kfold = KFold(5, shuffle=True, random_state = 1)
epochs = 5
dropout_val = 0.1


accuracies = []
for train, test in kfold.split(train_dataset):
  test_model = test_drop_model(dropout_val)
  x_train, y_train, x_val, y_val = train_dataset[train], train_onehot_labels[train], train_dataset[test], train_onehot_labels[test]
  history = test_model.fit(x_train, y_train, epochs = epochs, batch_size = 32, validation_data=(x_val, y_val), verbose = 1)
  _, acc = test_model.evaluate(x_val, y_val, verbose=0)
  print('Fold accuracy: ', acc*100.0)
  accuracies.append(acc)
  dropout_val = dropout_val + 0.1

# Small LR-Adam Model

In [ ]:
def lr_adam_model():
  model = Sequential()
  model.add(Conv2D(32, kernel_size= 3, activation='relu', input_shape = (64,64,1)))
  model.add(BatchNormalization())
  model.add(Conv2D(32, kernel_size = 3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(32, kernel_size= 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(64, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size = 3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size= 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(128, kernel_size = 4, activation='relu'))
  model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(55, activation='softmax'))
  model.add(Reshape((5, 11)))

  optimizer = Adam(lr=0.0002, beta_1=0.5)
  model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
  return model

# Max Pooling Model

In [ ]:
def pooling_model(input_shape):
  model = Sequential()
  model.add(Conv2D(32, kernel_size= 3, activation='relu', input_shape = input_shape))
  model.add(BatchNormalization())
  model.add(Conv2D(32, kernel_size = 3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Conv2D(32, kernel_size= 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(64, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
  model.add(Conv2D(64, kernel_size = 3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size= 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))
  model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))


  model.add(Conv2D(128, kernel_size = 4, activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(55, activation='softmax'))
  model.add(Reshape((5, 11)))
  opt = SGD(lr=0.01, momentum=0.9)
  model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
  return model

In [ ]:
model_2 = pooling_model((64, 64, 1))
train_x2, test_x2, train_y2, test_y2 = train_test_split(train_dataset, train_onehot_labels, test_size = 0.25)
history = model_2.fit(train_x2, train_y2, epochs=10, batch_size = 32, validation_data=(test_x2, test_y2))

predictions = model_2.predict(test_dataset)
result = evaluate(predictions)

results_to_csv(result, 'pool_model-original_data')

# Medium-Deep Model

In [ ]:
def medium_deep_model(input_shape, lr=0.01, beta_1=0.9, beta_2=0.99, amsgrad=False):
  model = Sequential()
  model.add(Conv2D(32, kernel_size = 3, padding='same', activation='relu', input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(Conv2D(32, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(64, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(128, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, kernel_size = 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(32, kernel_size = 4, padding='same', activation='relu'))
  model.add(BatchNormalization())

  model.add(Flatten())
  model.add(Dense(55, activation='softmax'))
  model.add(Reshape((5, 11)))

  opt = Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2, amsgrad=amsgrad)
  model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
  return model

In [ ]:
# tuning for betas
betas = [0.5, 0.75, 0.8, 0.9, 0.93, 0.95, 0.99]
b_accuracies = []
b_meanvalacc = []
b_histories = []

for beta in betas:
  print('-----------> evaluating b = ', beta)
  acc, accuracies, histories = kfold_evaluation(medium_deep_model, beta, (20,64,1), new_cut, train_onehot_labels, n_epochs=10)
  b_accuracies.append(accuracies)
  b_meanvalacc.append(acc)
  b_histories.append(histories)

In [ ]:
lr_val_losses = []
for hist in lr_histories:
  lr_val_losses.append(np.mean(hist[0].history['loss']))
plt.title('Average Fold Loss for Betas')
plt.plot(betas, lr_val_losses)
plt.show()

In [ ]:
plt.title('Average Fold Validation Accuracy for Betas')
plt.plot(learning_rates, lr_val_losses)
plt.show()

In [ ]:
# tuning for learning rate
learning_rates = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.8, 0.9]
lr_accuracies = []
lr_meanvalacc = []
lr_histories = []

for lr in learning_rates:
  print('-----------> evaluating lr = ', lr)
  acc, accuracies, histories = kfold_evaluation_lr(medium_deep_model, lr, (20,64,1), new_cut, train_onehot_labels, n_epochs=10)
  lr_accuracies.append(accuracies)
  lr_meanvalacc.append(acc)
  lr_histories.append(histories)

In [ ]:
lr_val_losses = []
for hist in lr_histories1:
  lr_val_losses.append(np.mean(hist[0].history['loss']))

plt.title('Average Fold Loss for Learning Rates')
plt.plot(learning_rates1, lr_val_losses)
plt.show

In [ ]:
plt.title('Average Fold Validation Accuracy for Learning Rates')
plt.plot(learning_rates1, lr_val_losses)
plt.show()

In [ ]:
# new_cut dataset no change kfold training - 98.9
model_k, accuracies, histories = kfold_training(medium_deep_model, (20, 64, 1), new_cut, train_onehot_labels, n_epochs=10)

predictions = model_k.predict(new_cut_test)
result = evaluate(predictions)

results_to_csv(result, 'medium_deep_cut_kfold')

In [ ]:
# new_cut dataset no change - 99.67 val acc, submitted 98 test acc
model_2 = medium_deep_model((20, 64, 1))
train_x2, test_x2, train_y2, test_y2 = train_test_split(new_cut, train_onehot_labels, test_size = 0.25)
history = model_2.fit(train_x2, train_y2, epochs=10, batch_size = 32, validation_data=(test_x2, test_y2), callbacks=[early, checkpoint])

predictions = model_2.predict(new_cut_test)
result = evaluate(predictions)

results_to_csv(result, 'medium_deep-cut')

In [ ]:
# new_cut and shearing - 
datagen = ImageDataGenerator(shear_range = 0.3)

# model_4 = medium_deep_model((20, 64, 1))
train_x, test_x, train_y, test_y = train_test_split(new_cut, train_onehot_labels, shuffle=True, test_size=0.25)
history = model_4.fit(datagen.flow(train_x, train_y, batch_size=32), epochs=5, validation_data=(test_x, test_y))

predictions = model_4.predict(new_cut_test)
result = evaluate(predictions)

results_to_csv(result, 'result_deep_shear_cut_pred')

# OpenCV

In [ ]:
individual_numbers = []
individual_targets = []

for i in range(56000):
  original_img = train_dataset[i].copy()
  ret, thresh = cv2.threshold(train_dataset[i].copy(), 23, 255, cv2.THRESH_BINARY)
  contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  contours.sort(key=lambda ct: cv2.boundingRect(ct)[0])

  j = 0
  for c in contours:
    if j > 4:
      continue
    x, y, width, height = cv2.boundingRect(c)
    if width < 4 and height < 4:
      continue

    segmented_digit = original_img[y:y+height, x:x+width]

    resized_segmented_digit = cv2.resize(segmented_digit, (18, 18))
    padded_digit = np.pad(resized_segmented_digit, ((5,5),(5,5)), "constant", constant_values=0) # This part is to add 5 pixels in 4 directions to make it 28x28
    individual_numbers.append(padded_digit)
    individual_targets.append(train_labels[i][j])

    j+=1

train, test = tf.keras.datasets.mnist.load_data(path="mnist.npz")
new_train_dataset = np.concatenate([np.array(individual_numbers), train[0], test[0]])
train_onehot_labels = to_categorical(individual_targets)
new_train_labels = np.array(train_onehot_labels)

new_labels = []
for onehot in new_train_labels:
  new_labels.append(onehot[:-1])
new_train_labels = np.array(new_labels)

new_train_labels = np.concatenate([new_train_labels, np.array(to_categorical(train[1])), np.array(to_categorical(test[1]))])

In [ ]:
# Open CV CNN model
def cv_model(input_shape):
  model = Sequential()
  model.add(Conv2D(32, kernel_size= 3, activation='relu', input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(Conv2D(32, kernel_size = 3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(32, kernel_size= 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(64, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size = 3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size= 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(128, kernel_size = 4, activation='relu'))
  model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
  return model

In [ ]:
# pre-process test dataset so that it fits our model
individual_numbers_test = []
batches_length = []

for i in range(len(test_dataset)):
  original_img = test_dataset[i].copy()
  ret, thresh = cv2.threshold(test_dataset[i].copy(), 23, 255, cv2.THRESH_BINARY)
  contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  contours.sort(key=lambda ct: cv2.boundingRect(ct)[0])

  j = 0
  for c in contours:
    if j > 4:
      continue
    x, y, width, height = cv2.boundingRect(c)
    if width < 4 and height < 4:
      continue

    segmented_digit = original_img[y:y+height, x:x+width]

    resized_segmented_digit = cv2.resize(segmented_digit, (18, 18))
    padded_digit = np.pad(resized_segmented_digit, ((5,5),(5,5)), "constant", constant_values=0) # This part is to add 5 pixels in 4 directions to make it 28x28
    individual_numbers_test.append(padded_digit)

    j+=1
  batches_length.append(j)

new_test_dataset = np.array(individual_numbers_test)

In [ ]:
# predictions and output to csv
def evaluate(yhat):
  results = []
  for y in yhat:
    results.append(np.argmax(y))
  return results

new_train_dataset = new_train_dataset.reshape(new_train_dataset.shape[0],new_train_dataset.shape[1], new_train_dataset.shape[2], 1)
cv_mod = cv_model((28,28,1))
train_x2, test_x2, train_y2, test_y2 = train_test_split(new_train_dataset, new_train_labels, test_size = 0.10)
history = cv_mod.fit(train_x2, train_y2, epochs=15, batch_size = 32, validation_data=(test_x2, test_y2))
predictions = cv_mod.predict(new_test_dataset.reshape(new_test_dataset.shape[0],new_test_dataset.shape[1], new_test_dataset.shape[2], 1))
results = evaluate(predictions)

concatenated_predictions = []
i = 0
for bl in batches_length:
  single_prediction = []
  for j in range(i, i + bl):
    single_prediction.append(results[j])
  for k in range(len(single_prediction), 5):
    single_prediction.append(10)
  concatenated_predictions.append(single_prediction)
  i = i + bl

results = pd.Series(concatenated_predictions, name="Label")
submission = pd.concat([pd.Series(range(0, 14000), name="Id"), results], axis=1)
submission.to_csv('/content/drive/MyDrive/openCV_og.csv', index=False)

# Deep Model

In [ ]:
def deep_model(input_shape, lr=0.01, beta_1=0.9, beta_2=0.99, amsgrad=False):
  model = Sequential()
  model.add(Conv2D(32, kernel_size = 3, padding='same', activation='relu', input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(Conv2D(32, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(64, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(128, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, kernel_size = 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(256, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(256, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(256, kernel_size = 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(512, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(5125, kernel_size = 3, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(512, kernel_size = 5, strides=2, padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(32, kernel_size = 4, padding='same', activation='relu'))
  model.add(BatchNormalization())

  model.add(Flatten())
  model.add(Dense(55, activation='softmax'))
  model.add(Reshape((5, 11)))

  opt = Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2, amsgrad=amsgrad)
  model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
  return model

##Other

In [ ]:
datagen = ImageDataGenerator(shear_range=0.3,
                             data_format='channels_last',
                             fill_mode='nearest')
img = test_dataset[10]
img = img.reshape((1, 64, 64, 1))
aug_iter = datagen.flow(img, batch_size=32)
# datagen_visualize(aug_iter)
# bat = next(aug_iter)
fig, ax = plt.subplots(nrows= 1, ncols=9, figsize=(15,15))
for i in range(9):
  image = next(aug_iter)[0] #.astype('uint8')
  ax[i].imshow(image)
  ax[i].axis('off')